In [1]:
import polars as pl
from polars import Config

In [2]:
Config.set_fmt_str_lengths(100)

polars.config.Config

# ACLARACIÓN IMPORTANTE: Trabajé los dataframes en archivos separados, asi que para combinarlos los estoy importando.

In [29]:
#Importando Atenciones, leyó bien los tipos de datos asi que lo dejaré así
Atenciones=pl.read_excel(
    source="./Atenciones/Atenciones.xlsx",
    sheet_name="Hoja1",
    engine="xlsx2csv",
)

In [30]:
#Importando Tickets, leyó mal los tipos de datos en las fechas
Tickets=pl.read_excel(
    source="./Tickets/Tickets.xlsx",
    sheet_name="Hoja1",
    engine="xlsx2csv",
    
)

In [27]:
#Corrigiendo el tipo de datos en las columnas fechas
Tickets=Tickets.select(
    'TicketID',
    'Ubicacion',
    'Service Desk',
    'Estado',
    pl.col("Fecha Creacion").cast(pl.Date),
    pl.col("Fecha Termino").cast(pl.Date),
    pl.col("Fecha Cierre").cast(pl.Date),
    'Agencia',
    'AgenciaID',
    pl.col("Fecha Real Fin").cast(pl.Date),
    'Dias Cierre',
    'Grupo Dias'

)

# Combinar Base Ticket y Atenciones

In [31]:
Tickets.join(
    Atenciones,
    on="TicketID",
    how="inner"
).shape

(22717, 15)

In [32]:
#Uniendo los DF con join
Resultado=Tickets.join(
    Atenciones,
    on="TicketID",
    how="inner"
).select(
    'TicketID',
    'AgenciaID',
    'Agencia',
    'Service Desk',
    'Estado',
    'Fecha Creacion',
    'Fecha Cierre',
    "Grupo Dias",
    pl.col("Tipo de Ticket").alias("Tipo Ticket"),
    pl.col("Costo Atencion").alias("Costo"),
)

In [33]:
Resultado.head()

TicketID,AgenciaID,Agencia,Service Desk,Estado,Fecha Creacion,Fecha Cierre,Grupo Dias,Tipo Ticket,Costo
str,i64,str,str,str,str,str,str,str,f64
"""WO0000005584689""",191076,"""LAS MALVINAS""","""Zona Centro""","""Cerrado""","""2023-09-08""","""2023-09-20""","""7 a 15 dias""","""Variable""",null
"""WO0000004544943""",191053,"""URB SAN FELIPE COMAS""","""Zona Centro""","""Rechazado""","""2022-10-18""",null,"""0 a 3 dias""","""Rechazado""",100.0
"""WO0000004545246""",191035,"""PUENTE PIEDRA""","""Zona Centro""","""Rechazado""","""2022-10-18""",null,"""0 a 3 dias""","""Rechazado""",100.0
"""WO0000004559125""",191087,"""EL PINO""","""Zona Centro""","""Rechazado""","""2022-10-24""",null,"""0 a 3 dias""","""Rechazado""",100.0
"""WO0000004707477""",191044,"""LAS ALONDRAS""","""Zona Centro""","""Rechazado""","""2022-11-08""",null,null,"""Rechazado""",100.0


### Exportando a un excel

In [34]:
Resultado.write_excel(
    workbook="Consolidado.xlsx",
    worksheet="Atenciones",
    table_style="Table Style Light 16",
    dtype_formats={pl.Date:"dd/mm/yyyy"},
    float_precision=2,
    autofit=True
)